In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import csv
import math

In [26]:
filename = "2019_after_LDA.csv"
k = 18

In [22]:
df = pd.read_csv(filename, encoding="ANSI")
df = df[['date', 'press', 'category', 'headline', 'url', 'text', 'img', 'N_list', 'modified_N_list', 'topics']]
df = df.dropna()
news_topic = df['topics'].tolist()
df

,date,press,category,headline,url,text,img,N_list,modified_N_list,topics
0,2019/01/01,한겨레,정치,문희상 “임시정부 100년…새 역사 시작하는 새해 될 것”,http://www.hani.co.kr/arti/politics/politics_g...,"문희상 국회의장이 2019년 새해를 “대한민국 임시정부 100년, 임시의정원 100...",['http://img.hani.co.kr/imgdb/resize/2018/1231...,"[['문희상', '국회', '의장', '새해', '대한민국 임시정부', '임시', ...","['문희상', '국회', '의장', '새해', '대한민국 임시정부', '임시', '...","문희상, 국회, 해"
1,2019/01/01,세계일보,"경제,지역",경남 승강기안전공단 지역과 상생 동반성장 활동,http://www.segye.com/content/html/2018/12/31/2...,경남 한국승강기안전공단이 지역과 상생하며 동반성장하는 다양한 활동에 나서고 있다. ...,[],"[['경남', '한국', '승강기', '안전', '공단', '지역', '상생', '...","['경남', '한국', '엘리베이터', '안전', '공단', '지역', '상생', ...","지역, 지역_인재, 채용, 공단, 포함, 신규"
2,2019/01/01,중앙일보,정치,문 대통령 신년사 “한 분 한 분의 삶 나아지도록 노력하겠다”,http://news.joins.com/article/olink/22844371,"문재인 대통령은 1일 “이 겨울, 더 따뜻하게 세상을 밝히라는 촛불의 마음 결코 잊...",['https://pds.joins.com/news/component/htmlpho...,"[['문재인 ', '대통령', '겨울', '세상', '촛불', '마음', '삶', ...","['문재인 ', '대통령', '겨울', '세상', '촛불', '마음', '삶', '...","문재인, 평화"
3,2019/01/01,동아일보,경제,"이마트, 올해 상반기 ‘한우 등심’ 초저가로 가격 동결",http://news.donga.com/3/all/20181231/93504466/3,올해 7월까지 ‘웻(Wet)에이징’ 숙성 한우 등심 33% 할인된 가격으로 동결바이...,['https://dimg.donga.com/wps/NEWS/IMAGE/2018/1...,"[['올해', '웻_이징', '숙성', '한우_등심', '할인', '가격', '동결...","['올해', '웻_이징', '숙성', '한우_등심', '할인', '가격', '동결'...","이마트, 숙성, 한우_등심, 한우, 한우_가격, 올해, 가격, 등급, 대비, 상승"
4,2019/01/01,국민일보,"사회,정치","“그 비서 몸매 좋던데”…국회 페미 “페미니즘=시대정신, 국회부터 바뀌어야”",http://news.kmib.co.kr/article/view.asp?arcid=...,지난해 8월 16일 국회 내에 신설된 익명 단체인 ‘국회 페미’가 최근 언론 인터뷰...,['http://image.kmib.co.kr/online_image/2019/01...,"[['지난해', '국회 ', '신설', '익명', '단체', '국회 페_미', '최...","['지난해', '국회 ', '신설', '익명', '단체', '국회 페_미', '최근...","여성, 국회, 남성, 국회 , 여성_의원, 국회 페_미, 직원, 주장, 남성_의원,..."
...,...,...,...,...,...,...,...,...,...,...
128646,2019/06/30,세계일보,"정치,국제",[WT논평]The reason why we must denuclearize Nort...,http://www.segye.com/content/html/2019/06/30/2...,By Robert R. Monroe (former director of the De...,[],"[['북한', '핵', '이유'], ['합의', '시바', '삐', '매듭', '필...","['북한', '핵', '이유', '합의', '시바', '삐', '매듭', '필요',...","핵, 핵무기, 글로벌, 북한, 이란, 나라, 불량_국가, 존재, 사용"
128647,2019/06/30,세계일보,"국제,정치","[사설] 日 경제보복 카드 우려, 정부는 전략적 대응 모색하길",http://www.segye.com/content/html/2019/06/30/2...,일본이 강제징용 판결에 대해 경제보복 카드를 꺼내든다고 한다. 일본 정부가 TV·스...,['//img.segye.com/content/image/2019/06/30/201...,"[['일본', '강제', '징용', '판결', '경제', '보복', '카드'], [...","['일본', '강제', '징용', '판결', '경제', '보복', '카드', '일본...","양국, 관계, 갈등, 보복_조치, 일본, 경제, 보복, 수출, 기업, 총리"
128648,2019/06/30,세계일보,정치,[사설] 84일 만의 국회 본회의… ‘완전한 정상화’ 서두를 때다,http://www.segye.com/content/html/2019/06/30/2...,국회가 지난 28일 자유한국당을 포함한 여야 5당이 모두 참석한 가운데 본회의를 열...,['//img.segye.com/content/image/2019/06/28/201...,"[['국회', '자유한국당', '포함', '여야', '당', '참석', '가운데',...","['국회', '자유한국당', '포함', '여야', '당', '참석', '가운데', ...","국회, 자유한국당, 여야, 처리, 합의, 본회의, 정상, 특위, 상임위, 경제"
128649,2019/06/30,세계일보,정치,"[사설] 남·북·미 정상 판문점 회동, 실질적 비핵화로 이어져야",http://www.segye.com/content/html/2019/06/30/2...,분단·냉전 상징서 역사적 만남 / 트럼프 “북·미 실무협상 할 것” / 北 비핵화 ...,['//img.segye.com/content/image/2019/06/30/201...,"[['분단', '냉전', '상징', '역사', '만남', '트럼프', '북_미 ',...","['분단', '냉전', '상징', '역사', '만남', '트럼프_대통령', '북_미...","핵, 트럼프_대통령, 협상, 김정은, 북_미 , 실무, 이번, 상징, 만남, 대통령"


In [27]:
# CountVectrizer로 토큰화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news_topic)

# l2 정규화
X = normalize(X)

# k-means 알고리즘 적용
print("waiting for KMeans")
kmeans = KMeans(n_clusters=k).fit(X)

# trained labels and cluster centers
labels = kmeans.labels_
centers = kmeans.cluster_centers_

# labels에 merge
df['labels'] = labels

df.to_csv("k_clustered_" + filename, encoding="utf-8-sig", mode='w', index=None)

waiting for KMeans
